# Import

In [2]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings(action='ignore') 

C:\Users\pc\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\pc\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\pc\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Fixed Random-Seed

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# Load Data

In [4]:
train_df = pd.read_csv('../open/train.csv')
test_df = pd.read_csv('../open/test.csv')

In [6]:
building_info = pd.read_csv("../open/building_info.csv")

# Data check

In [7]:
print(train_df.shape)
display(train_df)

(204000, 10)


,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08


In [8]:
print(test_df.shape)
display(test_df)

(16800, 7)


,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77
...,...,...,...,...,...,...,...
16795,100_20220831 19,100,20220831 19,22.5,0.0,0.9,84
16796,100_20220831 20,100,20220831 20,20.7,0.0,0.4,95
16797,100_20220831 21,100,20220831 21,20.2,0.0,0.4,98
16798,100_20220831 22,100,20220831 22,20.1,0.0,1.1,97


In [9]:
print(building_info.shape)
display(building_info)

(100, 7)


,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...
95,96,호텔및리조트,93314.00,60500.00,-,-,-
96,97,호텔및리조트,55144.67,25880.00,-,-,-
97,98,호텔및리조트,53578.62,17373.75,-,-,-
98,99,호텔및리조트,53499.00,40636.00,-,-,-


In [11]:
train_df[train_df.건물번호 == 1]

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
2035,1_20220824 19,1,20220824 19,25.5,NaN,2.9,63.0,0.1,0.19,2517.12
2036,1_20220824 20,1,20220824 20,25.0,NaN,2.6,66.0,0.0,0.01,2208.96
2037,1_20220824 21,1,20220824 21,24.6,NaN,1.3,67.0,NaN,NaN,1806.24
2038,1_20220824 22,1,20220824 22,24.1,NaN,2.1,70.0,NaN,NaN,1387.20


In [12]:
train_df[train_df.건물번호 == 2]

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
2040,2_20220601 00,2,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1170.36
2041,2_20220601 01,2,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1146.96
2042,2_20220601 02,2,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,1115.28
2043,2_20220601 03,2,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,1107.00
2044,2_20220601 04,2,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,1069.92
...,...,...,...,...,...,...,...,...,...,...
4075,2_20220824 19,2,20220824 19,25.5,NaN,2.9,63.0,0.1,0.19,1866.96
4076,2_20220824 20,2,20220824 20,25.0,NaN,2.6,66.0,0.0,0.01,1693.80
4077,2_20220824 21,2,20220824 21,24.6,NaN,1.3,67.0,NaN,NaN,1564.92
4078,2_20220824 22,2,20220824 22,24.1,NaN,2.1,70.0,NaN,NaN,948.96


In [13]:
test_df[test_df.건물번호 == 1]

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77
...,...,...,...,...,...,...,...
163,1_20220831 19,1,20220831 19,23.5,0.0,2.0,75
164,1_20220831 20,1,20220831 20,22.4,0.0,2.0,80
165,1_20220831 21,1,20220831 21,21.7,0.0,1.6,81
166,1_20220831 22,1,20220831 22,21.1,0.0,1.0,83


In [14]:
train_df.건물번호.nunique()

100

In [15]:
test_df.건물번호.nunique()

100

In [17]:
building_info.건물유형.unique()

array(['건물기타', '공공', '대학교', '데이터센터', '백화점및아울렛', '병원', '상용', '아파트', '연구소',
       '지식산업센터', '할인마트', '호텔및리조트'], dtype=object)

In [18]:
train_df.isnull().sum()

num_date_time         0
건물번호                  0
일시                    0
기온(C)                 0
강수량(mm)          160069
풍속(m/s)              19
습도(%)                 9
일조(hr)            75182
일사(MJ/m2)         87913
전력소비량(kWh)            0
dtype: int64

In [19]:
test_df.isnull().sum()

num_date_time    0
건물번호             0
일시               0
기온(C)            0
강수량(mm)          0
풍속(m/s)          0
습도(%)            0
dtype: int64

In [23]:
building_info['태양광용량(kW)'] = building_info['태양광용량(kW)'].replace('-',None)

In [25]:
building_info['ESS저장용량(kWh)'] = building_info['ESS저장용량(kWh)'].replace('-',None)

In [26]:
building_info['PCS용량(kW)'] = building_info['PCS용량(kW)'].replace('-',None)

In [27]:
building_info.isnull().sum()

건물번호             0
건물유형             0
연면적(m2)          0
냉방면적(m2)         0
태양광용량(kW)       64
ESS저장용량(kWh)    95
PCS용량(kW)       95
dtype: int64

In [30]:
print(train_df['전력소비량(kWh)'].max(),train_df['전력소비량(kWh)'].min())

25488.4 0.0
